# Chat Driver

An OpenAI Chat Completions API wrapper.

## Notebook setup

Run this cell to set the notebook up. Other sections can be run independently.

In [2]:
%reload_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
from azure.identity import aio, DefaultAzureCredential, get_bearer_token_provider, AzureCliCredential

from openai import AsyncAzureOpenAI, AzureOpenAI

import logging 
import json
from pathlib import Path

LOGGING = {
    "version": 1,
    "disable_existing_loggers": False,
    "formatters": {
        "json": {
            "()": "pythonjsonlogger.jsonlogger.JsonFormatter",
            "fmt": "%(asctime)s %(levelname)s %(name)s %(message)s",

        }
    },
}


# Set up structured logging to a file. All of the cells in this notebook use
# this logger. Find them at .data/logs.jsonl.
class JsonFormatter(logging.Formatter):
    def format(self, record) -> str:
        record_dict = record.__dict__
        log_record = {
            'timestamp': self.formatTime(record, self.datefmt),
            'level': record.levelname,
            'session_id': record_dict.get('session_id', None),
            'run_id': record_dict.get('run_id', None),
            'message': record.getMessage(),
            'data': record_dict.get('data', None),
            'module': record.module,
            'funcName': record.funcName,
            'lineNumber': record.lineno,
            'logger': record.name,
        }
        extra_fields = {
            key: value for key, value in record.__dict__.items() 
            if key not in ['levelname', 'msg', 'args', 'exc_info', 'funcName', 'module', 'lineno', 'name', 'message', 'asctime', 'session_id', 'run_id', 'data']
        }
        log_record.update(extra_fields)
        return json.dumps(log_record)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
modules = ['httpcore.connection', 'httpcore.http11', 'httpcore.sync.connection', 'httpx', 'openai', 'urllib3.connectionpool', 'urllib3.util.retry']
for module in modules:
    logging.getLogger(module).setLevel(logging.ERROR)
if logger.hasHandlers():
    logger.handlers.clear()
data_dir = Path('.data')
if not data_dir.exists():
    data_dir.mkdir()
handler = logging.FileHandler(data_dir / 'logs.jsonl')
handler.setFormatter(JsonFormatter())
logger.addHandler(handler)


load_dotenv()
credential = DefaultAzureCredential()

azure_openai_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT", ""),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION", ""),
    "max_retries": 2,
}
logger.info("Azure OpenAI configuration", extra=azure_openai_config)

async_client = AsyncAzureOpenAI(
    **azure_openai_config,
    azure_ad_token_provider=aio.get_bearer_token_provider(
        aio.AzureCliCredential(),
        "https://cognitiveservices.azure.com/.default",
    ),
)

client = AzureOpenAI(
    **azure_openai_config,
    azure_ad_token_provider=get_bearer_token_provider(
        AzureCliCredential(),
        "https://cognitiveservices.azure.com/.default",
    ),
)

model: str = azure_openai_config.get("azure_deployment", "gpt-4o")


## ChatCompletionsAPI usage

Azure/OpenAI's Chat Completions API is the fundamental building block of an AI assistant that uses the GPT model. 

- https://platform.openai.com/docs/api-reference/chat
- https://github.com/openai/openai-python/blob/main/api.md
- https://platform.openai.com/docs/api-reference/chat drivers

### Sync

In [3]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model=model,
)
print(completion.model_dump_json(indent=2))

{
  "id": "chatcmpl-ARRAZhqMa0AhQEzs0YAstZfa5CMm8",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "This is a test. How can I assist you further?",
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1731102659,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": "fp_d54531d9eb",
  "usage": {
    "completion_tokens": 12,
    "prompt_tokens": 12,
    "total_tokens": 24,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  }
}


### Async

In [4]:
message_event = await async_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model=model,
)
print(message_event)

ChatCompletion(id='chatcmpl-AQgaCid05i8dGgUHijQsvPAsMYbv4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1730923580, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_d54531d9eb', usage=CompletionUsage(completion_tokens=5, prompt_tokens=12, total_tokens=17, completion_tokens_details=None, prompt_tokens_details=None))


### Streaming

In [5]:
stream = await async_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model=model,
    stream=True,
)
async for chunk in stream:
    print(chunk.model_dump())

{'id': 'chatcmpl-AQgaEhy4ed19h72VpBu8nJBJE3g3L', 'choices': [{'delta': {'content': '', 'function_call': None, 'refusal': None, 'role': 'assistant', 'tool_calls': None}, 'finish_reason': None, 'index': 0, 'logprobs': None}], 'created': 1730923582, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion.chunk', 'service_tier': None, 'system_fingerprint': 'fp_d54531d9eb', 'usage': None}
{'id': 'chatcmpl-AQgaEhy4ed19h72VpBu8nJBJE3g3L', 'choices': [{'delta': {'content': 'This', 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': None, 'index': 0, 'logprobs': None}], 'created': 1730923582, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion.chunk', 'service_tier': None, 'system_fingerprint': 'fp_d54531d9eb', 'usage': None}
{'id': 'chatcmpl-AQgaEhy4ed19h72VpBu8nJBJE3g3L', 'choices': [{'delta': {'content': ' is', 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': None, 'index': 0, 'logprobs': None}], 'create

## OpenAI Helpers

### Standardized response handling

In [ ]:
from context import Context
from openai_client.errors import CompletionError, validate_completion
from openai_client.logging import make_completion_args_serializable, add_serializable_data
from openai_client.completion import message_content_from_completion

context = Context("conversation-id-1005")

# We use a metadata dictionary in our helpers to store information about the
# completion request.
metadata = {}

# This is just standard OpenAI completion request arguments.
completion_args = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": "You are a famous computer scientist. You are giving a talk at a conference. You are talking about the future of AI and how it will change the world. You are asked a questions by audience members and answer thoughtfully.",
        },
        {
            "role": "user",
            "content": "What is the future of AI?",
        }
    ],
}

# If we these completion args to logs and metadata, though, they need to be
# serializable. We have a helper for that.
metadata["completion_args"] = make_completion_args_serializable(completion_args)

# We have helpers for validating the response and handling exceptions in a
# standardized way. These ensure that logging happens and metadata is loaded up
# properly.
try:
    completion = await async_client.beta.chat.completions.parse(**completion_args)

    # This helper looks for any error-like situations (the model refuses to
    # answer, content filters, incomplete responses) and throws exceptions that
    # are handled by the next helper. The first argument is an identifier that
    # will be used for logs and metadata namespacing.
    validate_completion(completion)
    logger.debug("completion response.", extra=add_serializable_data(completion))
    metadata["completion"] = completion.model_dump()

except Exception as e:
    # This helper processes all the types of error conditions you might get from
    # the OpenAI API in a standardized way.
    completion_error = CompletionError(e)
    print(completion_error)
    print(completion_error.body)

else:
    # The message_string helper is used to extract the response from the completion
    # (which can get tedious).
    print(message_content_from_completion(completion))


The future of AI is both exciting and complex, with its trajectory shaped by advances in technology, ethical considerations, and societal needs. As we look ahead, several key themes emerge:

1. **Integration and Personalization**: AI will become increasingly integrated into our daily lives, driving personalized experiences. From healthcare to education, AI systems will tailor recommendations and interventions to individual needs, optimizing outcomes across various sectors.

2. **AI in Healthcare**: We can expect AI to revolutionize the healthcare industry by enhancing diagnostic accuracy, streamlining administrative processes, and developing personalized medicine. AI-driven tools could predict diseases, recommend treatments, and even aid in surgical procedures, ultimately improving patient care and reducing costs.

3. **Autonomous Systems**: Autonomous vehicles, drones, and robotic systems will become more prevalent, transforming industries like transportation, logistics, and manufactu

### Output types

#### JSON output

In [ ]:
from context import Context
from openai_client.errors import CompletionError, validate_completion

from openai_client.logging import make_completion_args_serializable, add_serializable_data
from openai_client.completion import message_content_dict_from_completion, JSON_OBJECT_RESPONSE_FORMAT

context = Context("conversation-id-1002")
metadata = {}
completion_args = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": "You are a famous computer scientist. You are giving a talk at a conference. You are talking about the future of AI and how it will change the world. You are asked a questions by audience members and return your answer as valid JSON like { \"thoughts\": <some thoughts>, \"answer\": <an answer> }.",
        },
        {
            "role": "user",
            "content": "What is the future of AI?",
        }
    ],
    "response_format": JSON_OBJECT_RESPONSE_FORMAT,
}
metadata["completion_args"] = make_completion_args_serializable(completion_args)
try:
    completion = await async_client.beta.chat.completions.parse(**completion_args)
    validate_completion(completion)
    metadata["completion"] = completion.model_dump()
except Exception as e:
    completion_error = CompletionError(e)
    metadata["completion_error"] = completion_error.body
    logger.error(completion_error.message, extra=add_serializable_data({"error": completion_error.body, "metadata": metadata}))
else:
    message = message_content_dict_from_completion(completion)
    print(json.dumps(message, indent=2))


{
  "thoughts": "AI is rapidly evolving and has the potential to transform virtually every industry. With advancements in machine learning, natural language processing, and robotics, AI will continue to play a crucial role in automating routine tasks and providing intelligent insights.",
  "answer": "The future of AI involves deeper integration into everyday life and industry. We will likely see AI systems becoming more autonomous, sophisticated, and seamlessly integrated into systems like healthcare, transportation, and personalized services. AI's future encompasses not only technological advancements but also ethical considerations and ensuring beneficial societal impacts."
}


#### Structured output

Any Pydantic BaseModel can be used as the "response_format" and OpenAI will try to load it up for you.

In [5]:
from context import Context
from pydantic import BaseModel
from typing import cast
from openai_client.errors import CompletionError, validate_completion
from openai_client.logging import add_serializable_data, make_completion_args_serializable


class Output(BaseModel):
    thoughts: str
    answer: str

context = Context("conversation-id-1002")
metadata = {}
completion_args = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": "You are a famous computer scientist. You are giving a talk at a conference. You are talking about the future of AI and how it will change the world. You are asked a questions by audience members and return your thoughtful answer.",
        },
        {
            "role": "user",
            "content": "What is the future of AI?",
        }
    ],
    "response_format": Output,
}

metadata["completion_args"] = make_completion_args_serializable(completion_args)
try:
    completion = await async_client.beta.chat.completions.parse(**completion_args)
    validate_completion(completion)
    metadata["completion"] = completion.model_dump()
except Exception as e:
    completion_error = CompletionError(e)
    metadata["completion_error"] = completion_error.body
    logger.error(completion_error.message, extra=add_serializable_data({"error": completion_error.body, "metadata": metadata}))
else:
    # The parsed message is in the `parsed` attribute.
    output = cast(Output, completion.choices[0].message.parsed)
    print(output.model_dump_json(indent=2))

    # Or you can just get the text of the message like usual.
    # print(completion.choices[0].message.content)


{
  "thoughts": "The future of AI is a convergence of technological advancement, ethical consideration, and societal adaptation. Its trajectory will be shaped by how well we manage its integration into daily life, ensuring it remains a tool to augment human capabilities rather than replace them entirely.",
  "answer": "The future of AI is incredibly promising and multifaceted, with potential developments across various domains. In healthcare, AI may revolutionize diagnostics and personalized medicine, allowing for earlier detection of diseases and tailored treatment plans. In the realm of transportation, we can expect more sophisticated autonomous vehicles, improving safety and efficiency on our roads. Moreover, AI could enhance environmental monitoring and response, aiding in the fight against climate change by optimizing energy use and predicting environmental changes.\n\nHowever, as we look to this future, it is crucial to address the ethical implications of AI. This includes ensuri

### Tools

#### Simple tool usage

The OpenAI chat completions API used the idea of "tools" to let the model request running a local tool and then processing the output. To use it, you need to create a JSON Schema representation of the function you want to use as a tool, check the response for the model requesting to run that function, run the function, and give the model the results of the function run for a final call.

While you can continue doing all of this yourself, our `complete_with_tool_calls` helper function makes this all easier for you.

Instead of generating JSON schema and executing functions yourself, you can use our `ToolFunctions` class to define the functions you want to be used.

In [34]:
from pydantic import Field
from openai_client.errors import CompletionError, validate_completion
from openai_client.tools import complete_with_tool_calls, ToolFunctions, ToolFunction


def square_the_number(number: int) -> int:
    """
    Return the square of the number.
    """
    return number * number


tool_functions = ToolFunctions([
    ToolFunction(square_the_number),
])

metadata = {}
completion_args = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": "You are an assistant.",
        },
        {
            "role": "user",
            "content": "What's the square of 53?",
        }
    ],
}

try:
    completion, new_messages = await complete_with_tool_calls(async_client, completion_args, tool_functions, metadata)
    validate_completion(completion)
except Exception as e:
    completion_error = CompletionError(e)
    metadata["completion_error"] = completion_error.body
    print(completion_error.message)
    print(completion_error.body)
    print(json.dumps(metadata, indent=2))
else:
    if completion:
        print(completion.choices[0].message.content)
        # print(json.dumps(metadata, indent=2))
    else:
        print("No completion returned.")


The square of 53 is 2809.


#### Structured tool inputs and output

You can use Pydantic models as input to tool function arguments.

You can also tell the model to respond with structured JSON or Pydantic model structures.

Here's an example of doing both things.

In [35]:
from pydantic import BaseModel
from typing import cast
from openai_client.errors import CompletionError, validate_completion
from openai_client.tools import complete_with_tool_calls, ToolFunctions, ToolFunction


class Input(BaseModel):
    zipcode: str

class Weather(BaseModel):
    description: str = Field(description="The weather description.")
    cloud_cover: float
    temp_c: float
    temp_f: float

def get_weather(input: Input) -> Weather:
    """Return the weather."""
    return Weather(description="Sunny", cloud_cover=0.2, temp_c=25.0, temp_f=77.0)

class Output(BaseModel):
    thoughts: str
    answer: str

metadata = {}
completion_args = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": "You are an assistant.",
        },
        {
            "role": "user",
            "content": "what is the weather in 90210?",
        }
    ],
    "response_format": Output,
}

functions = ToolFunctions([
    ToolFunction(get_weather),
])

try:
    completion, new_messages = await complete_with_tool_calls(async_client, completion_args, functions, metadata)
    validate_completion(completion)
except Exception as e:
    completion_error = CompletionError(e)
    metadata["completion_error"] = completion_error.body
    print(completion_error.message)
    print(completion_error.body)
    print(json.dumps(metadata, indent=2))
else:
    if completion:
        # The parsed message is in the `parsed` attribute.
        output = cast(Output, completion.choices[0].message.parsed)
        print(output.model_dump_json(indent=2))
    else:
        print("No completion returned.")

    # Or you can just get the text of the message like usual.
    # print(completion.choices[0].message.content)


{
  "thoughts": "The weather is typically sunny in Beverly Hills, especially considering the geographic location known for its warm climate.",
  "answer": "The weather in the 90210 area (Beverly Hills) is currently sunny with a temperature of 25°C (77°F). There is minimal cloud cover, at 20%."
}


### Tool functions with shadowed locals

If you want to make a local function available to be run as a chat completion
tool, you can. However, oftentimes, the local function might have some extra
arguments that you don't want the model to have to fill out for you in a tool
call. In this case, you can create a wrapper function that has the same
signature as the tool call and then calls the local function with the extra
arguments filled in.

In [36]:
from openai_client.errors import CompletionError, validate_completion
from openai_client.tools import complete_with_tool_calls, ToolFunctions, ToolFunction

# Here is the real function that does the work.
def real_square_the_number(number: int, binary: bool = True) -> str:
    """
    Return the square of the number.
    """
    if binary:
        return bin(number * number)
    return str(number * number)


# Here is the wrapper function that whose signature will be used as the tool
# call. You can just have it calls the real function with the extra arguments
# filled in.
def fish_calc(number: int) -> str:
    """
    Return the square of the number.
    """
    return real_square_the_number(number, binary=True)

# Add then just add wrapper to the tool functions you pass to the
# `complete_with_tool_calls` function. This is a way you can expose _any_
# function to be called by the model, but with the args you want the model to
# fill in!
tool_functions = ToolFunctions([
    ToolFunction(fish_calc),
])

metadata = {}
completion_args = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": "You are an assistant.",
        },
        {
            "role": "user",
            "content": "Run a fish calculation on 53 for me.",
        }
    ],
}

try:
    completion, new_messages = await complete_with_tool_calls(async_client, completion_args, tool_functions, metadata)
    validate_completion(completion)
except Exception as e:
    completion_error = CompletionError(e)
    metadata["completion_error"] = completion_error.body
    print(completion_error.message)
    print(completion_error.body)
    print(json.dumps(metadata, indent=2))
else:
    if completion:
        print(completion.choices[0].message.content)
        # print(json.dumps(metadata, indent=2))
    else:
        print("No completion returned.")


The result of the fish calculation on the number 53 is: `0b101011111001` in binary representation.


## OpenAI Chat Completion Driver (a.k.a "chat driver")

### OpenAI Assistants

The Azure/OpenAI Assistants API is newer, stateful API that splits an `assistant` from the data about a conversation `thread` that can be `run` against an `assistant`. Additionally, you can add `tools` to an assistant that enable the assistant to have more interactive capabilities. The tools currently available are:

- *Functions*: Registering local functions with the assistant so it knows it can call them before generating a response. This is a "hold on let me look that up for you" kind of interaction.
- *File Search* (formerly the retrieval plugin): Attach one or more files and they will be RAG-vectorized and available as content to the assistant.
- *Code Interpreter*: Run python code in a secure sandbox.

The Assistant API productized as OpenAI's `GPTs` product. The `GPT Builder` lets developers create and deploy GPTs assistants using a web interface.

### Chat Driver

But an "assistant" requires pretty strong "abstraction lock-in". This thing isn't really an assistant in the fullest sense... it's more like a "pseudo-assistant", but this confuses things. Let's just let the Chat Completion API be what it is and drive it as necessary as we create our assistants. Let's just wrap up the function calling bits (which, ultimately, can give you the other tools like Functions and File Search) in a simple-to-use GPT-like interface we'll call a *chat driver*.

The chat driver is meant to be used the exact way the Chat Completions API is... just easier.

Our chat driver provides:

- The ability to almost magically register functions to the function tool using a `FunctionRegistry`.
- Tracking of message history.
- Management of a `Context` object that can be used for session management and supply additional context to functions.
- Some prompt creation helpers.
- Other utilities... this is just meant to be an interface you can use to forget about all the api complexities.

#### Here is the simplest usage of a chat driver

Notice that a .data directory is created by default. This is where the conversation history is stored.

In [3]:
from chat_driver import ChatDriver, ChatDriverConfig

instructions = "You are a famous computer scientist. You are giving a talk at a conference. You are talking about the future of AI and how it will change the world. You are asked a questions by audience members."

chat_driver = ChatDriver(
    ChatDriverConfig(
        openai_client=async_client,
        model=model,
        instructions=instructions,
    ),
)

message_event = await chat_driver.respond("What is the future of AI?")
print(message_event.message)
# print(message_event.model_dump_json(indent=2))

The future of AI is both exciting and complex, with the potential to profoundly transform many aspects of our lives over the coming decades. Let's explore a few key areas where AI is expected to make significant impacts:

1. **Healthcare**: AI has the potential to revolutionize healthcare by improving diagnostics, personalizing treatment plans, and enhancing patient monitoring. Machine learning algorithms can analyze vast amounts of medical data to detect patterns that might be missed by human doctors, leading to earlier and more accurate diagnoses.

2. **Autonomous Systems**: From self-driving cars to drones, autonomous systems powered by AI are likely to become increasingly common. These systems can improve efficiency, safety, and accessibility in transportation and logistics, while also opening up new possibilities in areas like agriculture and disaster response.

3. **Work and Productivity**: AI can automate routine tasks, freeing up humans to focus on more complex and creative wor

#### You can register functions to chat drivers

Chat drivers will use any functions you give it as both OpenAI tool calls, and as commands.

With each response call, you can specify what type of response you want to have... string, dictionary, or Pydantic model.

In [5]:
from typing import Any, cast
from chat_driver import ChatDriver, ChatDriverConfig
from chat_driver import LocalMessageHistoryProvider
from pydantic import BaseModel, Field
from openai_client.tools import ToolFunctions, ToolFunction


# When an chat driver is created, it will automatically create a context with a
# session_id. Or, if you want to use a specific session_id, you can pass it as
# an argument. This is useful for scoping this chat driver instance to an
# external identifier.
session_id = "conversation-id-1002"


# Define tool functions for the chat driver.
def get_file_contents(file_path: str) -> str:
    """
    Return the contents of a file.

    Args:
    - file_path: The path to the file.
    """
    return "The purpose of life is to be happy."


def erase(name: str) -> str:
    """Erases a stored value."""
    return f"{context.session_id}: {name} erased"

def json_thing() -> dict[str, Any]:
    """Return json."""
    return {"key": "value"}

class Input(BaseModel):
    zipcode: str

class Weather(BaseModel):
    description: str = Field(description="The weather description.")
    cloud_cover: float
    temp_c: float
    temp_f: float

def get_weather(input: Input) -> Weather:
    """Return the weather."""
    return Weather(description="Sunny", cloud_cover=0.2, temp_c=25.0, temp_f=77.0)

# Define the chat driver.
instructions = "You are a helpful assistant."

all_funcs = [ get_file_contents, erase, json_thing, get_weather ]

chat_driver = ChatDriver(
    ChatDriverConfig(
        openai_client=async_client,
        model=model,
        instructions=instructions,
        # message_provider=message_provider,
        commands=all_funcs,  # Commands can be registered when instantiating the chat driver.
        functions=all_funcs,  # Functions can be registered when instantiating the chat driver.
    ),
)

# Let's clear the data from previous runs by using a custom message provider.
message_provider = cast(LocalMessageHistoryProvider, chat_driver.message_provider)
message_provider.delete_all()


# You can also use the `register_function` decorator to register a function.
@chat_driver.register_function_and_command
def echo(text: str) -> str:
    """Return the text."""
    return f"Echoing: {text}"


# You can also register functions manually.
chat_driver.register_function_and_command(get_file_contents)

# Let's see if the agent can respond.
message_event = await chat_driver.respond("Hi, my name is Paul.")
print()
print(message_event.message)

# Help command (shows command available).
message_event = await chat_driver.respond("/help")
print()
print(message_event.message)

# We can run any function or command directly.
message_event = await chat_driver.functions.echo("Echo this.")
print()
print(message_event)

# Let's see if the chat driver has the ability to run it's own registered function.
message_event = await chat_driver.respond("Please tell me what's in file 123.txt.")
print()
print(message_event.message)

# Stuctured output.
message_event = await chat_driver.respond("What is the weather in 90210?", response_format=Weather)
print()
print(message_event.message)

# Let's see the full response event.
# print()
# print(response.to_json())


Hello, Paul! How can I assist you today?

Commands:
echo(text: str): Return the text.
erase(name: str): Erases a stored value.
get_file_contents(file_path: str): Return the contents of a file.

Args:
- file_path: The path to the file.
get_weather(input: Input): Return the weather.
help(): Return this help message.
json_thing(): Return json.

Echoing: Echo this.

The content of "123.txt" is: "The purpose of life is to be happy." How else can I help you today?

{"description":"Sunny","cloud_cover":0.2,"temp_c":25.0,"temp_f":77.0}


## Chat with a chat driver

In [6]:
from chat_driver import ChatDriverConfig, ChatDriver
from context import Context
from openai_client.tools import ToolFunction, ToolFunctions


def get_file_contents(file_path: str) -> str:
    """Returns the contents of a file."""
    return "The purpose of life is to be happy."


def erase(name: str) -> str:
    """Erases a stored value."""
    return f"{session_id}: {name} erased"


def echo(value: str) -> str:  # noqa: F811
    """Echos a value as a string."""
    match value:
        case str():
            return value
        case list():
            return ", ".join(map(str, value))
        case dict():
            return json.dumps(value)
        case int() | bool() | float():
            return str(value)
        case _:
            return str(value)

# Define the chat driver.
chat_driver_config = ChatDriverConfig(
    openai_client=async_client,
    model=model,
    instructions="You are an assistant that has access to a sand-boxed Posix shell.",
    commands=[ get_file_contents, erase, echo ],
    functions=[ get_file_contents, erase, echo ],
)

chat_driver = ChatDriver(chat_driver_config)

# Note: Look in the .data directory for the logs, message history, and other data.

# Chat with the skill.
while True:
    message = input("User: ")
    if message == "":
        break
    print(f"User: {message}", flush=True)
    message_event = await chat_driver.respond(message)
    if message_event.metadata.get("error"):
        print(f"Error: {message_event.metadata.get('error')}")
        print(message_event.to_json())
        continue
    # You can print the entire message event! 
    # print(response.to_json())
    print(f"Assistant: {message_event.message}", flush=True)

User: Hi!
Assistant: Hello! How can I assist you today?
User: What's the capital of America?
Assistant: The capital of the United States of America is Washington, D.C.
User: No, I meant South America.
Assistant: South America is a continent composed of multiple countries, each with its own capital. Could you specify which country's capital you're interested in within South America?
User: Mexico.
Assistant: Mexico is actually part of North America. The capital of Mexico is Mexico City.
User: Brazil.
Assistant: The capital of Brazil is Brasília.


## Chat Driver with an Assistant Drive

In [ ]:
from io import BytesIO
from typing import Any, BinaryIO
from chat_driver import ChatDriverConfig, ChatDriver, ChatDriverConfig
from context import Context
from assistant_drive import Drive, DriveConfig, IfDriveFileExistsBehavior 

def get_drive_from_context(context):
    return Drive(DriveConfig(root=f".data/drive/{context.session_id}"))

def write_file_contents(file_path: str, contents: str) -> str:
    """Writes the contents to a file."""
    drive = get_drive_from_context(context)
    content_bytes: BinaryIO = BytesIO(contents.encode("utf-8"))
    drive.write(content_bytes, file_path, if_exists=IfDriveFileExistsBehavior.OVERWRITE)
    return f"{file_path} updated."

def read_file_contents(file_path: str) -> str:
    """Returns the contents of a file."""
    drive = get_drive_from_context(context)
    with drive.open_file(file_path) as file:
        return file.read().decode("utf-8")

functions = [write_file_contents, read_file_contents]

# Define the chat driver.
chat_driver_config = ChatDriverConfig(
    openai_client=async_client,
    model=model,
    instructions="You are an assistant that has access to a sand-boxed Posix shell.",
    commands=functions,
    functions=functions,
)

chat_driver = ChatDriver(chat_driver_config)

# Note: Look in the .data directory for the logs, message history, and other data.

# Chat with the skill.
while True:
    message = input("User: ")
    if message == "":
        break
    print(f"User: {message}", flush=True)
    message_event = await chat_driver.respond(message)
    # You can print the entire response event! 
    # print(response.to_json())
    print(f"Assistant: {message_event.message}", flush=True)